In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import geopandas as gpd

In [3]:
#importing project specific functions to match changes in EDA notebook
from model_prep import get_feature_name, model_transformer_train, model_transformer_test

In [4]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVC
from sklearn.preprocessing import FunctionTransformer
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score
from sklearn.utils.class_weight import compute_sample_weight

In [5]:
from dask import dataframe as dd
import joblib
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
cluster.scheduler, cluster.workers

C:\Users\bwils\anaconda3\envs\pumpitup\lib\site-packages\distributed\node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 63185 instead
  warnings.warn(


(<Scheduler: "tcp://127.0.0.1:63188" processes: 3 cores: 6>,
 {0: <Nanny: tcp://127.0.0.1:63213, threads: 2>,
  1: <Nanny: tcp://127.0.0.1:63207, threads: 2>,
  2: <Nanny: tcp://127.0.0.1:63209, threads: 2>})

In [6]:
model_data = pd.read_pickle('Data/model_data.pkl')

In [7]:
submit_X = pd.read_csv('Source_data/testset_values.csv')

In [8]:
model_data.columns

Index(['id', 'class', 'amount_tsh', 'funder', 'gps_height', 'installer',
       'longitude', 'latitude', 'basin', 'subvillage', 'region_code',
       'district_code', 'ward', 'population', 'public_meeting',
       'scheme_management', 'scheme_name', 'permit', 'extraction_type',
       'management', 'management_group', 'payment_type', 'water_quality',
       'quantity', 'source', 'waterpoint_type', 'lga_Njombe', 'year', 'month',
       'years_old', 'popbins'],
      dtype='object')

In [9]:
model_data.drop(columns=['id'], inplace=True)

In [10]:
model_data_new = model_data.loc[model_data['years_old']<5]

In [11]:
num_cols = ['gps_height', 'population']
cat_cols = ['basin', 'region_code', 'district_code', 'extraction_type', 'payment_type', 'water_quality', 'quantity', 
            'source', 'management', 'management_group', 'waterpoint_type',  'funder', 'installer', 'subvillage', 
            'ward', 'scheme_management', 'scheme_name', 'popbins']

In [12]:
scaler = StandardScaler()
ohe = OneHotEncoder(handle_unknown='ignore')
CT = ColumnTransformer(remainder='passthrough', transformers=[('scaler', scaler, num_cols),
                                              ('ohe', ohe, cat_cols)], verbose=True, sparse_threshold=0)

In [13]:
def prep_train(unprepped_values, class_col):
    #splits the data into a train and test set ensuring that transformations are based only on the train set assuming we 
    #haven't seen the test set before
    train_X, test_X, train_y, test_y = train_test_split(unprepped_values.drop(columns=[class_col]), 
                                                        unprepped_values[class_col], test_size=0.25, random_state=42)
    train_Xct = CT.fit_transform(train_X)
    
    test_Xct = CT.transform(test_X)

    cols = get_feature_name(CT)
    train_Xf = pd.DataFrame(train_Xct, columns=cols)
    cols = get_feature_name(CT)
    test_Xf = pd.DataFrame(test_Xct, columns=cols)
    
    return train_Xf, train_y, test_Xf, test_y   

In [14]:
def prep_submit(train_data, test_data):
    #prepares the submission test set as we do our learning set, without refitting to the unseen data
            
    submit_Xt, id_col = model_transformer_test(train_data, test_data)
    submit_Xct = CT.transform(submit_Xt)

    cols = get_feature_name(CT)
    submit_Xf = pd.DataFrame(submit_Xct, columns=cols)
    
    return submit_Xf, cols, id_col

In [15]:
train_Xf, train_y, test_Xf, test_y = prep_train(model_data_new, 'class')

[ColumnTransformer] ........ (1 of 3) Processing scaler, total=   0.0s
[ColumnTransformer] ........... (2 of 3) Processing ohe, total=   0.1s
[ColumnTransformer] ..... (3 of 3) Processing remainder, total=   0.0s


C:\Users\bwils\OneDrive\Documents\GitHub\Pump-it-up\model_prep\gfn.py:36: UserWarning: Transformer scaler (type StandardScaler) does not provide get_feature_names. Will return input column names if available
  warnings.warn("Transformer %s (type %s) does not "


## Random Forest

In [16]:
rf = RandomForestClassifier(n_estimators = 250, criterion='entropy', #max_samples=.75, #max_depth=16,  
                            class_weight='balanced_subsample', verbose=True)

with joblib.parallel_backend('dask'):
    rf.fit(train_Xf, train_y)
print(rf.score(test_Xf, test_y))

[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 239 out of 250 | elapsed:    6.3s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    6.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.8501035196687371


[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    0.2s finished


In [17]:
with joblib.parallel_backend('dask'):
    print(cross_val_score(rf, train_Xf, train_y, cv=5))

distributed.core - ERROR - Exception while handling op heartbeat_worker
Traceback (most recent call last):
  File "C:\Users\bwils\anaconda3\envs\pumpitup\lib\site-packages\distributed\core.py", line 497, in handle_comm
    result = handler(comm, **msg)
  File "C:\Users\bwils\anaconda3\envs\pumpitup\lib\site-packages\distributed\scheduler.py", line 3820, in heartbeat_worker
    ws._executing = {
  File "C:\Users\bwils\anaconda3\envs\pumpitup\lib\site-packages\distributed\scheduler.py", line 3821, in <dictcomp>
    parent._tasks[key]: duration for key, duration in executing.items()
KeyError: 'batch_of__parallel_build_trees_1_calls-0d368035e4af4012847619fe3231a238'


[0.82815735 0.81987578 0.84334023 0.8426501  0.8336784 ]


## Submission

In [18]:
values = pd.read_csv('Source_data/trainset_values.csv')
submit_X = pd.read_csv('Source_data/testset_values.csv')

In [19]:
submit_Xf, cols, id_col = prep_submit(values, submit_X)

C:\Users\bwils\anaconda3\envs\pumpitup\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bwils\OneDrive\Documents\GitHub\Pump-it-up\model_prep\gfn.py:36: UserWarning: Transformer scaler (type StandardScaler) does not provide get_feature_names. Will return input column names if available
  warnings.warn("Transformer %s (type %s) does not "


In [20]:
submit_Xf

,scaler__gps_height,scaler__population,ohe__x0_Internal,ohe__x0_Lake Nyasa,ohe__x0_Lake Rukwa,ohe__x0_Lake Tanganyika,ohe__x0_Lake Victoria,ohe__x0_Pangani,ohe__x0_Rufiji,ohe__x0_Ruvuma-Southern_Coast,...,ohe__x17_6,amount_tsh,longitude,latitude,public_meeting,permit,lga_Njombe,year,month,years_old
0,1.596733,0.072687,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,35.290799,-4.059696,1.0,1.0,0.0,2013.0,2.0,1.0
1,0.955785,0.038854,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,36.656709,-3.309214,1.0,1.0,0.0,2013.0,2.0,13.0
2,0.952782,0.361070,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,34.767863,-5.004344,1.0,0.0,0.0,2013.0,2.0,3.0
3,-0.998584,-0.041700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,38.058046,-9.418672,0.0,1.0,0.0,2013.0,1.0,26.0
4,0.491960,-0.347805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,500.0,35.006123,-10.950412,0.0,1.0,0.0,2013.0,3.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,-1.348329,-0.412248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,38.852669,-6.582841,1.0,1.0,0.0,2011.0,2.0,23.0
14846,0.368155,4.324325,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1000.0,37.451633,-5.350428,1.0,0.0,0.0,2011.0,3.0,17.0
14847,0.816187,-0.122254,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,34.739804,-4.585587,1.0,0.0,0.0,2013.0,3.0,3.0
14848,0.098684,-0.202808,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,35.432732,-10.584159,1.0,1.0,0.0,2013.0,2.0,4.0


In [21]:
submit_pred_rf = rf.predict(submit_Xf)
submit_pred_rf = pd.DataFrame(submit_pred_rf, columns=['status_group'])
submit_pred_rf.insert(0,value=id_col, column='id')
submit_pred_rf

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    1.6s finished


,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional
...,...,...
14845,39307,functional
14846,18990,functional
14847,28749,functional
14848,33492,functional


In [23]:
submit_pred_rf.to_csv(r'submissions/D/submission_E_rf', index=False)

In [24]:
model_data_new

,class,amount_tsh,funder,gps_height,installer,longitude,latitude,basin,subvillage,region_code,...,payment_type,water_quality,quantity,source,waterpoint_type,lga_Njombe,year,month,years_old,popbins
1,functional,1.813779,other,1399.00000,other,34.698766,-2.147466,Lake Victoria,other,20,...,never pay,soft,insufficient,rainwater_harvesting,communal standpipe,0,2013,3,3,3
2,functional,4.000000,other,686.00000,other,37.460664,-3.821329,Pangani,Majengo,21,...,per bucket,soft,enough,dam,communal standpipe multiple,0,2013,2,4,2
5,functional,2.090102,other,10.61625,DWE,39.172796,-4.765587,Pangani,other,4,...,per bucket,salty,enough,other,communal standpipe multiple,0,2011,3,2,1
10,functional,1.000000,other,62.00000,other,39.209518,-7.034139,Wami-Ruvu,other,60,...,never pay,salty,enough,machine_dbh,other,0,2011,2,0,3
13,functional,2.000000,other,1368.00000,other,37.092574,-3.181783,Pangani,other,3,...,monthly,soft,enough,spring,communal standpipe,0,2013,3,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59378,functional,2.000000,other,2101.00000,other,34.620387,-9.358405,Rufiji,other,11,...,monthly,soft,enough,spring,communal standpipe,1,2011,3,3,2
59385,functional,2.000000,other,1327.00000,other,33.951681,-2.021854,Lake Victoria,Mlimani,20,...,monthly,salty,enough,machine_dbh,hand pump,0,2012,11,1,2
59386,functional,1.893111,other,1786.00000,DWE,31.738789,-8.532013,Lake Rukwa,other,15,...,never pay,soft,enough,machine_dbh,hand pump,0,2011,8,2,4
59389,functional,2.803963,other,783.00000,DWE,30.646486,-7.365418,Lake Tanganyika,other,15,...,never pay,soft,enough,machine_dbh,hand pump,0,2011,8,2,5


## XGB

In [49]:
train_y = train_y.map({'functional':2, 'non functional':0, 'functional needs repair':1})

2980              non functional
5246                  functional
22659                 functional
39888             non functional
13361                 functional
                  ...           
20338                 functional
34426             non functional
23976    functional needs repair
39317             non functional
1888              non functional
Name: class, Length: 14850, dtype: object

In [47]:
xgb = XGBClassifier(use_label_encoder=False)
with joblib.parallel_backend('dask'):
    xgb.fit(train_Xf, train_y)


[00:01:38] WARNING: ..\src\learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.0

In [50]:
test_y = test_y.map({'functional':2, 'non functional':0, 'functional needs repair':1})

In [51]:
xgb.score(test_Xf, test_y)

0.7962289562289562

In [52]:
with joblib.parallel_backend('dask'):
    print(cross_val_score(xgb, train_Xf, train_y, cv=5))#

[0.7983165  0.79640853 0.79461279 0.7956229  0.78900112]


In [ ]:
y_pred = rf.predict(test_Xf)
cm=confusion_matrix(test_y, y_pred, labels=rf.classes_)
disp = ConfusionMatrixDisplay(cm, display_labels=rf.classes_)
plt.figure(figsize=(40,8))
disp.plot()
plt.tight_layout()
plt.show()

In [54]:
submit_pred_xgb = xgb.predict(submit_Xf)
submit_pred_xgb

array([2, 2, 2, ..., 2, 2, 0], dtype=int64)

In [55]:
preds = submit_pred_rf.copy()

In [56]:
preds.insert(2, value=submit_pred_xgb, column='xgb')

In [58]:
preds['status_group'] = preds['status_group'].map({'functional':2, 'non functional':0, 'functional needs repair':1})

## ADABoost

In [69]:
ada = AdaBoostClassifier(rf, n_estimators=100, random_state=42)
with joblib.parallel_backend('dask'):
    ada.fit(train_Xf, train_y)
ada.score(test_Xf, test_y)

[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   25.1s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   30.6s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    1.2s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    1.6s finished
[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   27.1s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   33.4s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: D

[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    1.8s finished
[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   24.9s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   30.3s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    1.2s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    1.7s finished
[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   24.8s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   31.5s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jo

[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    1.9s finished
[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   25.2s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   30.3s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    1.8s finished
[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   23.8s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   28.8s finished
[Parallel(n_jobs=8)]: Using backe

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    1.3s finished
[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   22.4s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    1.3s finished
[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done 250 out of 250 

[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   12.3s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   13.4s finished
[Parallel(n_jobs=8)]: Using backe

[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   11.1s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   12.2s finished
[Parallel(n_jobs=8)]: Using backe

[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   11.8s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   11.1s finished
[Parallel(n_jobs=8)]: Using backe

[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    3.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    4.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    5.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    5.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    3.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    3.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    4.5s finished
[Parallel(n_jobs=1)]: Us

[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    2.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    1.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    1.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    2.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    1.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    2.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    2.0s finished
[Parallel(n_jobs=1)]: Us

0.7955555555555556

In [70]:
preds.insert(3, value=ada.predict(submit_Xf), column='ada')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    3.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    3.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    4.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    4.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    4.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    4.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    1.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    1.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    1.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    3.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    1.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    2.5s finished
[Parallel(n_jobs=1)]: Us

In [72]:
preds_save = preds.copy()

In [ ]:
## ((preds['rf'] == preds['xgb']) and (preds['rf'] == preds['ada'])).sum()

In [89]:
condition = [((preds['rf'].item()==preds['xgb'].item()) | (preds['rf'].item() == preds['ada'].item())), 
             (preds['xgb'].item()==preds['ada'].item()), 
             ((preds['rf'].item()!= preds['xgb'].item()) & (preds['rf'].item()!=preds['ada'].item()))]
values = [preds['rf'], preds['xgb'], preds['rf']]

ValueError: can only convert an array of size 1 to a Python scalar

In [62]:
preds.rename(columns={'status_group':'rf'}, inplace=True)

In [81]:
xgbada = preds.loc[preds['xgb'] != preds['ada']]
xgbrf = preds.loc[preds['xgb'] != preds['rf']]
adarf = preds.loc[preds['ada'] != preds['rf']]

xgbrf.loc[xgbrf['id'].isin(adarf['id'])]

,id,rf,xgb,ada
0,50785,0,2,2
72,20691,0,2,2
84,22705,2,0,0
122,60335,2,1,1
141,74104,2,0,0
...,...,...,...,...
14624,26244,2,1,1
14636,31723,2,1,1
14651,13503,2,0,0
14760,15994,0,2,2


In [91]:
preds['status_group']=9
preds.loc[(preds['rf']==preds['xgb']) | (preds['rf'] == preds['ada']), 'status_group'] = preds['rf']
preds.loc[(preds['rf']!=preds['xgb']) & (preds['xgb']==preds['ada']), 'status_group'] = preds['xgb']
preds.loc[preds['status_group']==9, 'status_group'] = preds['rf']
preds

,id,rf,xgb,ada,status_group
0,50785,0,2,2,2
1,51630,2,2,2,2
2,17168,2,2,2,2
3,45559,0,0,0,0
4,49871,2,2,2,2
...,...,...,...,...,...
14845,39307,0,0,0,0
14846,18990,2,2,2,2
14847,28749,2,2,2,2
14848,33492,2,2,2,2


In [92]:
(preds['status_group']!= preds['rf']).sum()

297

In [93]:
sub_combo = preds.drop(columns=['rf', 'xgb', 'ada'])
sub_combo

,id,status_group
0,50785,2
1,51630,2
2,17168,2
3,45559,0
4,49871,2
...,...,...
14845,39307,0
14846,18990,2
14847,28749,2
14848,33492,2


In [95]:
sub_combo['status_group'] = sub_combo['status_group'].map({2:'functional', 0:'non functional', 1:'functional needs repair'})
sub_combo

,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional
...,...,...
14845,39307,non functional
14846,18990,functional
14847,28749,functional
14848,33492,functional


In [96]:
sub_combo.to_csv('Submissions/D/sub_combo.csv', index=False)

In [74]:
preds.to_csv('preds.csv')